In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

#change path to your dataset path
df = pd.read_json("data\\Sarcasm_Headlines_Dataset_v2.json", lines=True)
# df["sentence_length"] = df["headline"].str.split(" ").apply(len)
df = df.drop_duplicates()

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df  = train_test_split(temp_df, test_size=0.5, random_state=42)

In [3]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("microsoft/deberta-v3-large").to("cuda")

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-large and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Load model directly
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-large")

In [5]:
import torch
print(torch.cuda.is_available())  # Should print True if CUDA is enabled
print(torch.version.cuda)  # Should print the CUDA version if available

True
12.1


In [8]:
from datasets import Dataset,DatasetDict

# Convert to Hugging Face DatasetDict
dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "val": Dataset.from_pandas(val_df),
    "test": Dataset.from_pandas(test_df)
})

In [9]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [10]:
def tokenize_dataset(dataset):
    return tokenizer(dataset["headline"])
dataset = dataset.map(tokenize_dataset, batched=True)

Map:   0%|          | 0/22893 [00:00<?, ? examples/s]

Map:   0%|          | 0/2862 [00:00<?, ? examples/s]

Map:   0%|          | 0/2862 [00:00<?, ? examples/s]

In [11]:
print(dataset.column_names)
dataset = dataset.remove_columns(['article_link', 'sentence_length', '__index_level_0__', 'headline'])
dataset = dataset.rename_column("is_sarcastic", "label")
print(dataset.column_names)

{'train': ['is_sarcastic', 'headline', 'article_link', 'sentence_length', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'], 'val': ['is_sarcastic', 'headline', 'article_link', 'sentence_length', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['is_sarcastic', 'headline', 'article_link', 'sentence_length', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask']}
{'train': ['label', 'input_ids', 'token_type_ids', 'attention_mask'], 'val': ['label', 'input_ids', 'token_type_ids', 'attention_mask'], 'test': ['label', 'input_ids', 'token_type_ids', 'attention_mask']}


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="D:/deberta-v3-large-finetuned-sarcasm",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    push_to_hub=True,
)

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["val"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

C:\Users\Raygu\AppData\Local\Temp\ipykernel_17636\1167372532.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,0.368800
1000,0.312900
1500,0.265200
2000,0.230100
2500,0.222100
3000,0.158500
3500,0.099700
4000,0.090300
4500,0.093900
5000,0.080700


TrainOutput(global_step=5724, training_loss=0.17801665260107014, metrics={'train_runtime': 1582.7522, 'train_samples_per_second': 28.928, 'train_steps_per_second': 3.616, 'total_flos': 1759967901441048.0, 'train_loss': 0.17801665260107014, 'epoch': 2.0})

In [ ]:
#TESTING STARTS HERE

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("charaydes/deberta-v3-large-finetuned-sarcasm")
inputs = tokenizer(test_df["headline"].tolist(), padding=True, truncation=True, return_tensors="pt").to("cuda") 

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
from torch.utils.data import DataLoader, TensorDataset
# Create a DataLoader to batch inputs
batch_size = 16
dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"])
loader = DataLoader(dataset, batch_size=batch_size)

In [8]:
from transformers import AutoModelForSequenceClassification

all_preds = []
model = AutoModelForSequenceClassification.from_pretrained("charaydes/deberta-v3-large-finetuned-sarcasm").to("cuda") 
torch.cuda.empty_cache()
with torch.no_grad():
    for batch in loader:
        input_ids, attention_mask = [x.to("cuda") for x in batch]
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())

config.json:   0%|          | 0.00/887 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

In [9]:
from sklearn.metrics import classification_report

print(classification_report(test_df["is_sarcastic"].tolist(), all_preds))

              precision    recall  f1-score   support

           0       0.96      0.97      0.96      1484
           1       0.97      0.95      0.96      1378

    accuracy                           0.96      2862
   macro avg       0.96      0.96      0.96      2862
weighted avg       0.96      0.96      0.96      2862

